# SOM Neighbourhood Graph Visualization

This notebook implements radius- and knn-based Neighbourhood Graph visualizations. 

Students: 
- Alexander Falzberger (01273054)
- Sebastian Strumbelj (12007910)

# Implementation
First, we define a function that computes all neighbours for every input vector.

The input data must be specified as a numpy array, where each row represents a data sample.
The method is specified with a string that is either 'knn' or 'radius'.
Additionally, the parameters for the method can be specified, their default values are
k=3 for knn-based neighbours and r=0.25 for radius-based neighbours.

In [ ]:
import numpy as np
from scipy.spatial import distance_matrix

def compute_neighbours(input_data:np.ndarray, method:str, k=3, r=0.25) -> dict:
    # abort if method is not supported
    assert method in ['radius', 'knn']

    # create pairwise distances
    distances = distance_matrix(input_data, input_data)

    # we map each index of an input vector to all neighbour indices
    n_inputs = input_data.shape[0]
    result = {}
    for input_idx in range(n_inputs):
        if method == 'radius':
            # filter all elements that are in the radius neighborhood
            result[input_idx] = [idx for (idx, dist) in enumerate(distances[:, input_idx]) if dist <= r and idx != input_idx]

        elif method == 'knn':
            # take the k nearest neighbors which correspond to the k first entries in the sorted list
            sorted_distances = sorted(enumerate(distances[:, input_idx]), key=lambda idx_and_dist: idx_and_dist[1])
            knn = list(map(lambda idx_and_dist: idx_and_dist[0], sorted_distances))[:k+1]
            if input_idx in knn:
                # it can happen that this is not the case, e.g. when k=1 and there are 3 input vectors with the exact
                # same coordinates. Then we can simply continue, as this does not affect subsequent program logic.
                knn.remove(input_idx)
            result[input_idx] = knn

    return result

Next we implement a function that provides us with the coordinates of the lines that should be displayed in the
visualization. I.e. each line represents two neighbours in input space that are not mapped to the same SOM unit.
The function uses a trained SOM, the input data and the neighbour information (as calculated in `compute_neighbours`).

In [ ]:
def compute_graph_lines(som_weights: np.ndarray, x_dim:int, y_dim:int, input_data: np.ndarray, neighbours: dict) -> (list, list):
    x_coords, y_coords = [], []
    som_weights = np.array(som_weights).reshape((x_dim, y_dim, input_data.shape[1]), order='F')

    # first, calculate the winning SOM unit for every input vector
    n_inputs = input_data.shape[0]
    winners = {} # map input_vector index to winning SOM unit
    for input_idx in range(n_inputs):
        winner = np.argmin(np.sqrt(np.sum(np.power(som_weights - input_data[input_idx], 2), axis=2)))
        winners[input_idx] = winner

    # then, use those to determine the lines for the neighborhood graphs
    # iterate over all input vectors and add a line to their respective neighbours
    for (input_idx, neighbours) in neighbours.items():
        win_neuron = winners[input_idx]
        win_neuron_unravel = np.unravel_index(win_neuron, (x_dim, y_dim))
        for neighbour in neighbours:
            neighbour_neuron = winners[neighbour]
            if win_neuron != neighbour_neuron:
                neighbour_neuron_unravel = np.unravel_index(neighbour_neuron, (x_dim, y_dim))
                x_coords += [win_neuron_unravel[0], neighbour_neuron_unravel[0], None]
                y_coords += [win_neuron_unravel[1], neighbour_neuron_unravel[1], None]

    return x_coords, y_coords

Before we now show the first example of our neighbourhood graph visualization, we define a few helper functions
for visualizing SOMs based on a given list of weights and its dimensions. Most of these functions were provided
in the assignment's template, we adapted it as little as possible to implement the function `neighbourhood_graph`.

In [ ]:
import plotly.graph_objects as go
from ipywidgets import HBox


class SomViz:
    def __init__(self, weights, y:int, x:int, width=700, height=700):
        self.weights = weights
        self.y = y
        self.x = x
        self.width = width
        self.height = height

    # Plots a Neighbourhood Graph visualization for the SOM with which the SomViz object has been initialised.
    # If an existing FigureWidget is provided in the parameter 'base_vis', the visualization is added as an overlay.
    # Parameter 'method' must be either 'knn' or 'radius', their default parameter values are k=3 and r=0.25, respectively.
    def neighbourhood_graph(self, input_data: np.ndarray, method:str, k=3, r=0.25, base_vis: go.FigureWidget = None,
                            color:str = 'rgb(210, 210, 210)', width:int = 2,
                            title:str = 'Neighbourhood Graph Visualization'):
        neighbours = compute_neighbours(input_data, method, k, r)
        x_coords, y_coords = compute_graph_lines(self.weights, self.x, self.y, input_data, neighbours)
        plot = go.Scatter(x=x_coords, y=y_coords, mode='lines', line=dict(color=color, width=width), hoverinfo='none')

        if base_vis is None:
            return go.FigureWidget(plot, layout=go.Layout(width=self.width, height=self.height, title={'text':title, 'x':0.5}))
        else:
            # do neighbourhood graph visualization as an overlay if a base visualization is provided
            base_vis.add_trace(plot)
            return base_vis

    def umatrix(self, som_map=None, color="Viridis", interp = "best", title=""):
        um = np.zeros((self.y * self.x, 1))
        neuron_locs = list()
        for i in range(self.y):
            for j in range(self.x):
                neuron_locs.append(np.array([i, j]))
        neuron_distmat = distance_matrix(neuron_locs,neuron_locs)

        for i in range(self.y * self.x):
            neighbor_idxs = neuron_distmat[i] <= 1
            neighbor_weights = self.weights[neighbor_idxs]
            um[i] = distance_matrix(np.expand_dims(self.weights[i], 0), neighbor_weights).mean()

        if som_map is None : return self.plot(um.reshape((self.y, self.x)), color=color, interp=interp, title=title)
        else: som_map.data[0].z = um.reshape(self.y,self.x)

    def hithist(self, input_vectors: np.ndarray, som_map=None, color='RdBu', interp = "best", title=""):
        hist = [0] * self.x * self.y
        for v in input_vectors:
            position = np.argmin(np.sqrt(np.sum(np.power(self.weights - v, 2), axis=1)))
            hist[position] += 1

        if som_map is None : return self.plot(np.array(hist).reshape(self.y,self.x), color=color, interp=interp, title=title)
        else:  som_map.data[0].z = np.array(hist).reshape(self.y,self.x)

    def component_plane(self, som_map=None, component=0, color="Viridis", interp = "best", title=""):
        if som_map is None : return self.plot(self.weights[:,component].reshape(-1,self.x), color=color, interp=interp, title=title)
        else:  som_map.data[0].z = self.weights[:,component].reshape(-1, self.x)

    def sdh(self, input_vectors: np.ndarray, som_map=None, sdh_type=1, factor=1, color="Cividis", interp = "best", title=""):

        import heapq
        sdh_m = [0] *self.y *self.x

        cs=0
        for i in range(0,factor): cs += factor-i

        for vector in input_vectors:
            dist = np.sqrt(np.sum(np.power(self.weights - vector, 2), axis=1))
            c = heapq.nsmallest(factor, range(len(dist)), key=dist.__getitem__)
            if sdh_type == 1:
                for j in range(0,factor):  sdh_m[c[j]] += (factor-j)/cs # normalized
            if sdh_type == 2:
                for j in range(0,factor): sdh_m[c[j]] += 1.0/dist[c[j]] # based on distance
            if sdh_type == 3:
                dmin = min(dist)
                for j in range(0,factor): sdh_m[c[j]] += 1.0 - (dist[c[j]]-dmin)/(max(dist)-dmin)

        if som_map is None : return self.plot(np.array(sdh_m).reshape(-1,self.x), color=color, interp=interp, title=title)
        else: som_map.data[0].z = np.array(sdh_m).reshape(-1,self.x)

    def project_data(self, input_vectors: np.ndarray, som_m=None):
        data_y = []
        data_x = []
        for v in input_vectors:
            position =np.argmin(np.sqrt(np.sum(np.power(self.weights - v, 2), axis=1)))
            x,y = position % self.x, position // self.x
            data_x.extend([x])
            data_y.extend([y])

        if som_m is not None : som_m.add_trace(go.Scatter(x=data_x, y=data_y, mode = "markers", marker_color=dict(color='rgba(255, 255, 255, 0.8)')))

    def time_series(self, input_vectors: np.ndarray, som_m=None, wsize=50): #not tested
        data_y = []
        data_x = [i for i in range(0,len(idata))]

        data_x2 = []
        data_y2 = []

        qmin = np.Inf
        qmax = 0

        ps = []
        for v in input_vectors:
            matrix = np.sqrt(np.sum(np.power(self.weights - v, 2), axis=1))
            position = np.argmin(matrix)
            qerror = matrix[position]
            if qmin>qerror: qmin = qerror
            if qmax<qerror: qmax = qerror
            ps.append((position, qerror))

        markerc=[]
        for v in ps:
            data_y.extend([v[0]])
            rez = v[1]/qmax

            markerc.append('rgba(0, 0, 0, '+str(rez)+')')

            x,y = v[0] % self.x, v[0] // self.x
            if    x==0: y = np.random.uniform(low=y, high=y+.1)
            elif  x==self.y-1: y = np.random.uniform(low=y-.1, high=y)
            elif  y==0: x = np.random.uniform(low=x, high=x+.1)
            elif  y==self.x-1: x = np.random.uniform(low=x-.1, high=x)
            else: x,y = np.random.uniform(low=x-.1, high=x+.1), np.random.uniform(low=y-.1, high=y+.1)

            data_x2.extend([x])
            data_y2.extend([y])

        ts_plot = go.FigureWidget(go.Scatter(x=[], y=[], mode = "markers", marker_color=markerc, marker=dict(colorscale='Viridis', showscale=True, color=np.random.randn(500))))
        ts_plot.update_xaxes(range=[0, wsize])


        ts_plot.data[0].x, ts_plot.data[0].y = data_x, data_y
        som_m.add_trace(go.Scatter(x=data_x2, y=data_y2, mode = "markers",))

        som_m.layout.height = 500
        ts_plot.layout.height = 500
        som_m.layout.width = 500
        ts_plot.layout.width = 1300

        return HBox([go.FigureWidget(som_m), go.FigureWidget(ts_plot)])

    def plot(self, matrix, color="Viridis", interp = "best", title=""):
        return go.FigureWidget(go.Heatmap(z=matrix, zsmooth=interp, showscale=False, colorscale=color),
                               layout=go.Layout(width=self.width, height=self.height,title={'text': title, 'x':0.5}))

Below is the (adapted) template code provided in the assignment for parsing SOMs from the SOMToolBox.

In [ ]:
import gzip
import pandas as pd

class SOMToolBoxParser:
    @staticmethod
    def read_weight_file(filename: str) -> (pd.DataFrame, int, int, int):
        if filename[-3:len(filename)] == '.gz':
            with gzip.open(filename, 'rb') as file:
                df, dim_vec, dim_x, dim_y = SOMToolBoxParser._read_vector_file_to_df(file)
        else:
            with open(filename, 'rb') as file:
                df, dim_vec, dim_x, dim_y = SOMToolBoxParser._read_vector_file_to_df(file)

        return df.astype('float64'), dim_vec, dim_x, dim_y


    @staticmethod
    def _read_vector_file_to_df(file) -> (pd.DataFrame, int, int, int):
        df = None
        dim_x, dim_y, dim_vec, position = 0, 0, 0, 0
        for byte in file:
            line = byte.decode('UTF-8')
            # get meta-data from vector file
            if line.startswith('$'):
                split = line.split(' ')
                param, val = split[0], split[1]
                if param == '$XDIM':
                    dim_x = int(val)
                elif param == '$YDIM':
                    dim_y = int(val)
                elif param == '$VEC_DIM':
                    dim_vec = int(val)
                if dim_x != 0 and dim_y != 0 and dim_vec != 0:
                    df = pd.DataFrame(index=range(0, dim_y * dim_x), columns=range(0, dim_vec))
            # fill dataframe with values
            else:
                if df is None:
                     raise ValueError('Weight file has missing dimensional information.')
                split = line.split(' ')
                try:
                    df.values[position] = list(np.array(split[0:dim_vec]).astype(float))
                    position += 1
                except:
                    raise ValueError('The input-vector file does not match its unit-dimension.')
        return df, dim_vec, dim_x, dim_y

# Evaluation Report - Example Visualizations

Below we visualize the neighborhood graphs on the Iris [1], Chainlink [2] and 10Clusters [2] datasets with different parameters to verify the correctness of our implementation.

## Visualization Examples on Iris Dataset

In [ ]:
# Example visualization with iris dataset
idata, idim, idata_x, idata_y = SOMToolBoxParser.read_weight_file('data/iris/iris.vec')
smap, sdim, smap_x, smap_y = SOMToolBoxParser.read_weight_file('data/iris/iris_40_20.wgt.gz')

In [ ]:
# Visualization
graphs = [] 
for k in [1,10]:
    viz_SOMToolBox = SomViz(smap.values.reshape(-1, sdim), smap_y, smap_x, 500, 500)
    um = viz_SOMToolBox.umatrix(color='viridis', interp=False, title=f'KNN (k={k}) U-matrix SOMToolBox')
    neighbourhood_graph = viz_SOMToolBox.neighbourhood_graph(idata.values, 'knn', k=k, base_vis=um)
    graphs.append(neighbourhood_graph)

HBox(graphs)

In [ ]:
# Visualization
w, h = 380, 380

graphs = []
for r in [0.1, 0.4, 0.8]:
    viz_SOMToolBox = SomViz(smap.values.reshape(-1, sdim), smap_y, smap_x, w, h)
    um = viz_SOMToolBox.umatrix(color='viridis', interp=False, title=f"Radius (r={r}) U-matrix SOMToolBox")
    neighbourhood_graph = viz_SOMToolBox.neighbourhood_graph(idata.values, 'radius', r=r, base_vis=um)
    graphs.append(neighbourhood_graph)
    
HBox(graphs)

## Visualization Examples on ChainLink Dataset

In [ ]:
# Example visualization with chainlink dataset
idata, idim, idata_x, idata_y = SOMToolBoxParser.read_weight_file('data/chainlink/chainlink.vec')
smap, sdim, smap_x, smap_y = SOMToolBoxParser.read_weight_file('data/chainlink/Chainlink_40_20.wgt.gz')

In [ ]:
# Visualization
graphs = [] 
for k in [1,10]:
    viz_SOMToolBox = SomViz(smap.values.reshape(-1, sdim), smap_y, smap_x, 500, 500)
    um = viz_SOMToolBox.umatrix(color='viridis', interp=False, title=f'KNN (k={k}) U-matrix SOMToolBox')
    neighbourhood_graph = viz_SOMToolBox.neighbourhood_graph(idata.values, 'knn', k=k, base_vis=um)
    graphs.append(neighbourhood_graph)

HBox(graphs)

In [ ]:
# Visualization
w, h = 380, 380

graphs = []
for r in [0.03, 0.1, 0.2]:
    viz_SOMToolBox = SomViz(smap.values.reshape(-1, sdim), smap_y, smap_x, w, h)
    um = viz_SOMToolBox.umatrix(color='viridis', interp=False, title=f"Radius (r={r}) U-matrix SOMToolBox")
    neighbourhood_graph = viz_SOMToolBox.neighbourhood_graph(idata.values, 'radius', r=r, base_vis=um)
    graphs.append(neighbourhood_graph)
    
HBox(graphs)

## Visualization Examples on 10 Clusters Dataset

In [ ]:
# Example visualization with chainlink dataset
idata, idim, idata_x, idata_y = SOMToolBoxParser.read_weight_file('data/10clusters/10clusters.vec')
smap, sdim, smap_x, smap_y = SOMToolBoxParser.read_weight_file('data/10clusters/10Clusters_40_20.wgt.gz')

In [ ]:
# Visualization
graphs = [] 
for k in [1,10]:
    viz_SOMToolBox = SomViz(smap.values.reshape(-1, sdim), smap_y, smap_x, 500, 500)
    um = viz_SOMToolBox.umatrix(color='viridis', interp=False, title=f'KNN (k={k}) U-matrix SOMToolBox')
    neighbourhood_graph = viz_SOMToolBox.neighbourhood_graph(idata.values, 'knn', k=k, base_vis=um)
    graphs.append(neighbourhood_graph)

HBox(graphs)

In [ ]:
# Visualization
w, h = 380, 380

graphs = []
for r in [0.03, 0.1, 0.5]:
    viz_SOMToolBox = SomViz(smap.values.reshape(-1, sdim), smap_y, smap_x, w, h)
    um = viz_SOMToolBox.umatrix(color='viridis', interp=False, title=f"Radius (r={r}) U-matrix SOMToolBox")
    neighbourhood_graph = viz_SOMToolBox.neighbourhood_graph(idata.values, 'radius', r=r, base_vis=um)
    graphs.append(neighbourhood_graph)
    
HBox(graphs)

# Evaluation - Comparison

In the following section, we will evaluate our implementation using the Chainlink dataset, and the 10-clusters dataset. For the images, run the whole notebook or check the pdf submission.

## Plot Generator

The following is just there to generate plots for the comparison shown in "Results". All other plots are taken from the examples above.

In [ ]:
idata, idim, idata_x, idata_y = SOMToolBoxParser.read_weight_file('data/chainlink/chainlink.vec')
smap, sdim, smap_x, smap_y = SOMToolBoxParser.read_weight_file('data/chainlink/Chainlink_100_60.wgt.gz')

In [ ]:
# Visualization
graphs = [] 
for k in [1,3,8]:
    viz_SOMToolBox = SomViz(smap.values.reshape(-1, sdim), smap_y, smap_x, 400, 400)
    um = viz_SOMToolBox.umatrix(color='viridis', interp=False, title=f'KNN (k={k}) U-matrix SOMToolBox')
    neighbourhood_graph = viz_SOMToolBox.neighbourhood_graph(idata.values, 'knn', k=k, base_vis=um)
    graphs.append(neighbourhood_graph)

HBox(graphs)

In [ ]:
# Visualization
graphs = [] 
for r in [0.1, 0.2, 0.3]:
    viz_SOMToolBox = SomViz(smap.values.reshape(-1, sdim), smap_y, smap_x, 400, 400)
    um = viz_SOMToolBox.umatrix(color='viridis', interp=False, title=f'Radius (r={r}) U-matrix SOMToolBox')
    neighbourhood_graph = viz_SOMToolBox.neighbourhood_graph(idata.values, 'radius', r=r, base_vis=um)
    graphs.append(neighbourhood_graph)

HBox(graphs)

In [ ]:
smap, sdim, smap_x, smap_y = SOMToolBoxParser.read_weight_file('data/chainlink/Chainlink_40_20.wgt.gz')

In [ ]:
# Visualization
graphs = [] 
for k in [1,3,8]:
    viz_SOMToolBox = SomViz(smap.values.reshape(-1, sdim), smap_y, smap_x, 400, 400)
    um = viz_SOMToolBox.umatrix(color='viridis', interp=False, title=f'KNN (k={k}) U-matrix SOMToolBox')
    neighbourhood_graph = viz_SOMToolBox.neighbourhood_graph(idata.values, 'knn', k=k, base_vis=um)
    graphs.append(neighbourhood_graph)

HBox(graphs)

In [ ]:
# Visualization
graphs = [] 
for r in [0.1, 0.2, 0.3]:
    viz_SOMToolBox = SomViz(smap.values.reshape(-1, sdim), smap_y, smap_x, 400, 400)
    um = viz_SOMToolBox.umatrix(color='viridis', interp=False, title=f'Radius (r={r}) U-matrix SOMToolBox')
    neighbourhood_graph = viz_SOMToolBox.neighbourhood_graph(idata.values, 'radius', r=r, base_vis=um)
    graphs.append(neighbourhood_graph)

HBox(graphs)

In [ ]:
idata, idim, idata_x, idata_y = SOMToolBoxParser.read_weight_file('data/10clusters/10clusters.vec')
smap, sdim, smap_x, smap_y = SOMToolBoxParser.read_weight_file('data/10clusters/10Clusters_100_60.wgt.gz')

In [ ]:
# Visualization
graphs = [] 
for k in [1,3,8]:
    viz_SOMToolBox = SomViz(smap.values.reshape(-1, sdim), smap_y, smap_x, 400, 400)
    um = viz_SOMToolBox.umatrix(color='viridis', interp=False, title=f'KNN (k={k}) U-matrix SOMToolBox')
    neighbourhood_graph = viz_SOMToolBox.neighbourhood_graph(idata.values, 'knn', k=k, base_vis=um)
    graphs.append(neighbourhood_graph)

HBox(graphs)

In [ ]:
# Visualization
graphs = [] 
for r in [0.1, 0.2, 0.3]:
    viz_SOMToolBox = SomViz(smap.values.reshape(-1, sdim), smap_y, smap_x, 400, 400)
    um = viz_SOMToolBox.umatrix(color='viridis', interp=False, title=f'Radius (r={r}) U-matrix SOMToolBox')
    neighbourhood_graph = viz_SOMToolBox.neighbourhood_graph(idata.values, 'radius', r=r, base_vis=um)
    graphs.append(neighbourhood_graph)

HBox(graphs)

In [ ]:
smap, sdim, smap_x, smap_y = SOMToolBoxParser.read_weight_file('data/10clusters/10Clusters_40_20.wgt.gz')

In [ ]:
# Visualization
graphs = [] 
for k in [1,3,8]:
    viz_SOMToolBox = SomViz(smap.values.reshape(-1, sdim), smap_y, smap_x, 400, 400)
    um = viz_SOMToolBox.umatrix(color='viridis', interp=False, title=f'KNN (k={k}) U-matrix SOMToolBox')
    neighbourhood_graph = viz_SOMToolBox.neighbourhood_graph(idata.values, 'knn', k=k, base_vis=um)
    graphs.append(neighbourhood_graph)

HBox(graphs)

In [ ]:
# Visualization
graphs = [] 
for r in [0.1, 0.2, 0.3]:
    viz_SOMToolBox = SomViz(smap.values.reshape(-1, sdim), smap_y, smap_x, 400, 400)
    um = viz_SOMToolBox.umatrix(color='viridis', interp=False, title=f'Radius (r={r}) U-matrix SOMToolBox')
    neighbourhood_graph = viz_SOMToolBox.neighbourhood_graph(idata.values, 'radius', r=r, base_vis=um)
    graphs.append(neighbourhood_graph)

HBox(graphs)